## tifftools reproducability test

See next section for generating tiff header from scratch, this is just experimental code with source data

In [61]:
import os

os.system('tifftools dump test.tiff')

-- test.tiff --
Header: 0x4949 <little-endian> <ClassicTIFF>
Directory 0: offset 8 (0x8)
  ImageWidth 256 (0x100) LONG: 6464
  ImageLength 257 (0x101) LONG: 4852
  BitsPerSample 258 (0x102) SHORT: 8
  Compression 259 (0x103) SHORT: 1 (None 1 (0x1))
  Photometric 262 (0x106) SHORT: 1 (MinIsBlack 1 (0x1))
  StripOffsets 273 (0x111) LONG: 122
  RowsPerStrip 278 (0x116) LONG: 4852
  StripByteCounts 279 (0x117) LONG: 31363328
  PlanarConfig 284 (0x11C) SHORT: 1 (Chunky 1 (0x1))


0

In [62]:
cmd = ('tifftools set '
      f'--set ImageWidth:LONG 0 '
      f'--set ImageLength:LONG 0 '
      f'--set RowsPerStrip:LONG 0 '
      f'--set StripByteCounts:LONG 0 '
       '--set BitsPerSample:SHORT 8 '
       '--set Compression:SHORT 1 '
       '--set Photometric:SHORT 1 '
       '--set PlanarConfig:SHORT 1 '
       '--littleendian --ifdsfirst -y test.tiff recons.tiff')
os.system(cmd)
os.system('tifftools dump recons.tiff')

-- recons.tiff --
Header: 0x4949 <little-endian> <ClassicTIFF>
Directory 0: offset 8 (0x8)
  ImageWidth 256 (0x100) LONG: 0
  ImageLength 257 (0x101) LONG: 0
  BitsPerSample 258 (0x102) SHORT: 8
  Compression 259 (0x103) SHORT: 1 (None 1 (0x1))
  Photometric 262 (0x106) SHORT: 1 (MinIsBlack 1 (0x1))
  StripOffsets 273 (0x111) LONG: 122
  RowsPerStrip 278 (0x116) LONG: 0
  StripByteCounts 279 (0x117) LONG: 0
  PlanarConfig 284 (0x11C) SHORT: 1 (Chunky 1 (0x1))


0

In [63]:
with open('recons.tiff', 'rb') as f:
    header = f.read()

with open('20240320022317714.Raw', 'rb') as f:
    raw = f.read()

with open('recons.tiff', 'wb') as f:
    f.write(header)
    f.write(raw)

In [64]:
dims = (6464, 4852)

cmd = ('tifftools set '
      f'--set ImageWidth:LONG {dims[0]} '
      f'--set ImageLength:LONG {dims[1]} '
      f'--set RowsPerStrip:LONG {dims[1]} '
      f'--set StripByteCounts:LONG {dims[0] * dims[1]} '
       '--littleendian --ifdsfirst -y recons.tiff')
os.system(cmd)
os.system('tifftools dump recons.tiff')

-- recons.tiff --
Header: 0x4949 <little-endian> <ClassicTIFF>
Directory 0: offset 8 (0x8)
  ImageWidth 256 (0x100) LONG: 6464
  ImageLength 257 (0x101) LONG: 4852
  BitsPerSample 258 (0x102) SHORT: 8
  Compression 259 (0x103) SHORT: 1 (None 1 (0x1))
  Photometric 262 (0x106) SHORT: 1 (MinIsBlack 1 (0x1))
  StripOffsets 273 (0x111) LONG: 122
  RowsPerStrip 278 (0x116) LONG: 4852
  StripByteCounts 279 (0x117) LONG: 31363328
  PlanarConfig 284 (0x11C) SHORT: 1 (Chunky 1 (0x1))


0

In [65]:
with open('recons.tiff', 'rb') as f:
    recons = f.read()

with open('test.tiff', 'rb') as f:
    original = f.read()

len(original), len(recons), recons == original

(31363450, 31363450, True)

In [98]:
os.remove('header.tiff')
os.remove('recons.tiff')

## Generate header from scratch

Here we generate a tiff header from scratch using dimensions of the Raw images (in our case 6464x4852)

In [99]:
import os
dims = (6464, 4852)

cmd = ('tifftools set '
      f'--set ImageWidth:LONG 0 '
      f'--set ImageLength:LONG 0 '
      f'--set RowsPerStrip:LONG 0 '
      f'--set StripByteCounts:LONG 0 '
       '--set BitsPerSample:SHORT 8 '
       '--set Compression:SHORT 1 '
       '--set PlanarConfig:SHORT 1 '
      # Set CFA metadate for raw image
       '--set Photometric:SHORT 32803 ' # CFA
       '--set CFARepeatPatternDim:SHORT 2,2 '
       '--set CFAPattern:BYTE 0,1,1,2 '
       '--set DNGVersion:BYTE 1,4,0,0 '
       '--littleendian --ifdsfirst -y header.bin header.tiff')
os.system(cmd)
os.system('tifftools dump header.tiff')

-- header.tiff --
Header: 0x4949 <little-endian> <ClassicTIFF>
Directory 0: offset 8 (0x8)
  ImageWidth 256 (0x100) LONG: 0
  ImageLength 257 (0x101) LONG: 0
  BitsPerSample 258 (0x102) SHORT: 8
  Compression 259 (0x103) SHORT: 1 (None 1 (0x1))
  Photometric 262 (0x106) SHORT: 32803 (CFA 32803 (0x8023))
  StripOffsets 273 (0x111) LONG: 158
  RowsPerStrip 278 (0x116) LONG: 0
  StripByteCounts 279 (0x117) LONG: 0
  PlanarConfig 284 (0x11C) SHORT: 1 (Chunky 1 (0x1))
  CFARepeatPatternDim 33421 (0x828D) SHORT: <2> 2 2
  CFAPattern 33422 (0x828E) BYTE: <4> 0 1 1 2
  DNGVersion 50706 (0xC612) BYTE: <4> 1 4 0 0


0

In [100]:
# set the dimensions for the raw image (this will raise warnings since we haven't set any actual data)
cmd = ('tifftools set '
      f'--set ImageWidth:LONG {dims[0]} '
      f'--set ImageLength:LONG {dims[1]} '
      f'--set RowsPerStrip:LONG {dims[1]} '
      f'--set StripByteCounts:LONG {dims[0] * dims[1]} '
       '--littleendian --ifdsfirst -y header.tiff')
os.system(cmd)
os.system('tifftools dump header.tiff')

-- header.tiff --
Header: 0x4949 <little-endian> <ClassicTIFF>
Directory 0: offset 8 (0x8)
  ImageWidth 256 (0x100) LONG: 6464
  ImageLength 257 (0x101) LONG: 4852
  BitsPerSample 258 (0x102) SHORT: 8
  Compression 259 (0x103) SHORT: 1 (None 1 (0x1))
  Photometric 262 (0x106) SHORT: 32803 (CFA 32803 (0x8023))
  StripOffsets 273 (0x111) LONG: 0
  RowsPerStrip 278 (0x116) LONG: 4852
  StripByteCounts 279 (0x117) LONG: 31363328
  PlanarConfig 284 (0x11C) SHORT: 1 (Chunky 1 (0x1))
  CFARepeatPatternDim 33421 (0x828D) SHORT: <2> 2 2
  CFAPattern 33422 (0x828E) BYTE: <4> 0 1 1 2
  DNGVersion 50706 (0xC612) BYTE: <4> 1 4 0 0


0

In [101]:
with open('header.tiff', 'rb') as f:
    header = f.read()

header_len = len(header)
print(header_len)

with open('../tulane_images/source/20240320022317714.Raw', 'rb') as f:
    raw = f.read()

# write concat a raw image to the header, this is needed to properly set the StripOffsets tag
with open('header.tiff', 'wb') as f:
    f.write(header)
    f.write(raw)

158


In [102]:
os.system('tifftools dump header.tiff')

-- header.tiff --
Header: 0x4949 <little-endian> <ClassicTIFF>
Directory 0: offset 8 (0x8)
  ImageWidth 256 (0x100) LONG: 6464
  ImageLength 257 (0x101) LONG: 4852
  BitsPerSample 258 (0x102) SHORT: 8
  Compression 259 (0x103) SHORT: 1 (None 1 (0x1))
  Photometric 262 (0x106) SHORT: 32803 (CFA 32803 (0x8023))
  StripOffsets 273 (0x111) LONG: 0
  RowsPerStrip 278 (0x116) LONG: 4852
  StripByteCounts 279 (0x117) LONG: 31363328
  PlanarConfig 284 (0x11C) SHORT: 1 (Chunky 1 (0x1))
  CFARepeatPatternDim 33421 (0x828D) SHORT: <2> 2 2
  CFAPattern 33422 (0x828E) BYTE: <4> 0 1 1 2
  DNGVersion 50706 (0xC612) BYTE: <4> 1 4 0 0


0

In [103]:
cmd = ('tifftools set '
      f'--set StripOffsets:LONG {header_len} '
       '--littleendian --ifdsfirst -y header.tiff')
os.system(cmd)
os.system('tifftools dump header.tiff')

-- header.tiff --
Header: 0x4949 <little-endian> <ClassicTIFF>
Directory 0: offset 8 (0x8)
  ImageWidth 256 (0x100) LONG: 6464
  ImageLength 257 (0x101) LONG: 4852
  BitsPerSample 258 (0x102) SHORT: 8
  Compression 259 (0x103) SHORT: 1 (None 1 (0x1))
  Photometric 262 (0x106) SHORT: 32803 (CFA 32803 (0x8023))
  StripOffsets 273 (0x111) LONG: 158
  RowsPerStrip 278 (0x116) LONG: 4852
  StripByteCounts 279 (0x117) LONG: 31363328
  PlanarConfig 284 (0x11C) SHORT: 1 (Chunky 1 (0x1))
  CFARepeatPatternDim 33421 (0x828D) SHORT: <2> 2 2
  CFAPattern 33422 (0x828E) BYTE: <4> 0 1 1 2
  DNGVersion 50706 (0xC612) BYTE: <4> 1 4 0 0


0

In [104]:
with open('header.tiff', 'rb') as f:
    header = f.read()

with open('header.bin', 'wb') as f:
    f.write(recons[:header_len])

os.system('tifftools dump header.bin')

-- header.bin --
Header: 0x4949 <little-endian> <ClassicTIFF>
Directory 0: offset 8 (0x8)
  ImageWidth 256 (0x100) LONG: 6464
  ImageLength 257 (0x101) LONG: 4852
  BitsPerSample 258 (0x102) SHORT: 8
  Compression 259 (0x103) SHORT: 1 (None 1 (0x1))
  Photometric 262 (0x106) SHORT: 32803 (CFA 32803 (0x8023))
  StripOffsets 273 (0x111) LONG: 158
  RowsPerStrip 278 (0x116) LONG: 4852
  StripByteCounts 279 (0x117) LONG: 31363328
  PlanarConfig 284 (0x11C) SHORT: 1 (Chunky 1 (0x1))
  CFARepeatPatternDim 33421 (0x828D) SHORT: <2> 2 2
  CFAPattern 33422 (0x828E) BYTE: <4> 0 1 1 2
  DNGVersion 50706 (0xC612) BYTE: <4> 1 4 0 0


0